# 1. [코사인 유사도_Flavor 기준]

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def find_top_similar_drinks(input_drink_name, csv_file_path):
    # CSV 파일을 Pandas DataFrame으로 읽기
    df = pd.read_csv(csv_file_path, header=None)  # 컬럼 이름이 없다고 지정

    # 입력된 술 이름으로부터 해당 술의 인덱스 찾기
    target_index = None
    for index, row in df.iterrows():
        if row.iloc[1] == input_drink_name:
            target_index = index
            break

    if target_index is None:
        return "술 이름을 찾을 수 없습니다."

    # 입력된 술의 특징 벡터 추출 (15, 16, 17, 18, 19 열 데이터)
    target_features = df.iloc[target_index, 14:19].values.reshape(1, -1)

    # 출력 입력 술의 특징 벡터
    print("Input Drink Features:")
    print(target_features)

    # 다른 술들의 특징 벡터 추출 (입력 술을 제외한 모든 술)
    other_features = df.drop(index=target_index).iloc[:, 14:19].values

    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(target_features, other_features)

    # 가장 높은 유사도를 가진 술 2개의 인덱스 찾기
    similar_indices = cosine_sim[0].argsort()[:-3:-1]

    # 가장 유사도가 높은 2개 술의 인덱스와 유사도 계산
    top_similar_drinks_indices = similar_indices
    top_similar_drinks_data = df.drop(index=target_index).iloc[top_similar_drinks_indices]
    similarities = [cosine_sim[0][index] for index in top_similar_drinks_indices]

    # 결과 데이터에 유사도 추가
    top_similar_drinks_data['유사도 값'] = similarities

    return top_similar_drinks_data


In [15]:
input_drink_name = '감사'
csv_file_path = '/content/drive/MyDrive/CJ올리브네트웍스 해커톤/DATA/술담화/CSV/alcohol_with_new_category_en.csv'

find_top_similar_drinks(input_drink_name, csv_file_path)

Input Drink Features:
[[4 3 2 -1 3]]


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,유사도 값
24,chungju,매화깊은밤,https://www.sooldamhwa.com/damhwaMarket/detail...,images/chungju_13.jpg,매실의 달콤함에 빠져버린 아름다운 밤이에요~,4.9,71,살균약주,8.00%,375ml,...,진한 풍미의 음식과 잘 어울려요,직접 만든 매실청이 들어갔어요,"차돌박이 숙주 볶음,시저 샐러드,백순대 볶음",4,3,2,-1,3,"bulgogi,salad,sundaebokkeum",1.000000
49,chungju,우포,https://www.sooldamhwa.com/damhwaMarket/detail...,images/chungju_18.jpg,창녕의 특산물 양파가 들어간 달콤한 술이에요,4.7,49,살균약주,12.00%,500ml,...,달콤 짭짤한 양념이 배어있는 음식과 즐겨보세요,국내 최초 양파를 넣어 만든 술,"양념 치킨,간장 불고기,고구마 맛탕",3,2,2,-1,2,"chicken,bulgogi,sweet_potato",0.990044


이렇게 유사도 값을 출력할 수 있습니다. flavor 만 가지고 유사도 측정했기때문에 tastePoint나 foodPoint의 유사도도 측정해보고 위의 결과값과 동일하게 한다면, 출력되는 유사도 값 중 제일 높게 나온 값을 기준으로 정하면 되지 않을까 생각해봤습니다..!


##현재 기준의 결과값으로 2번째 기준, **0.990044**이 나왔습니다.

# 2. [코사인 유사도_tastPoint & comment 기준]

In [29]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def find_top_similar_drinks2(input_drink_name, csv_file_path):
    # CSV 파일을 Pandas DataFrame으로 읽기
    df = pd.read_csv(csv_file_path, header=None)  # 컬럼 이름이 없다고 지정

    # 입력된 술 이름으로부터 해당 술의 인덱스 찾기
    try:
        target_index = df[df.iloc[:, 1] == input_drink_name].index[0]
    except IndexError:
        return "술 이름을 찾을 수 없습니다."

    # 입력된 술의 특징 벡터 추출 (5, 11 열 데이터)
    input_features = ' '.join(df.iloc[target_index, [4, 10]].astype(str))
    print(input_features)

    # 다른 술들의 특징 벡터 추출 (입력 술을 제외한 모든 술)
    other_features = df.drop(index=target_index).iloc[:, [4, 10]].apply(lambda x: ' '.join(x), axis=1)

    # TfidfVectorizer를 사용하여 텍스트 데이터를 벡터화
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix_input = tfidf_vectorizer.fit_transform([input_features])
    tfidf_matrix_other = tfidf_vectorizer.transform(other_features)

    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(tfidf_matrix_input, tfidf_matrix_other)

    # 가장 높은 유사도를 가진 술 2개의 인덱스 찾기
    similar_indices = cosine_sim[0].argsort()[::-1][:2]

    # 가장 유사도가 높은 2개 술의 인덱스와 유사도 계산
    top_similar_drinks_indices = similar_indices
    top_similar_drinks_data = df.drop(index=target_index).iloc[top_similar_drinks_indices]
    similarities = [cosine_sim[0][index] for index in top_similar_drinks_indices]

    # 결과 데이터에 유사도 추가
    top_similar_drinks_data['유사도'] = similarities

    return top_similar_drinks_data


In [30]:
input_drink_name = '감사'
csv_file_path = '/content/drive/MyDrive/CJ올리브네트웍스 해커톤/DATA/술담화/CSV/alcohol_with_new_category_en.csv'

find_top_similar_drinks2(input_drink_name, csv_file_path)

소중한 분들게 감사한 마음을 담아 달콤한 카스테라가 떠오르는 부드러운 술이에요


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,유사도
71,soju,가무치 소주 25%,https://www.sooldamhwa.com/damhwaMarket/detail...,images/soju_86.jpg,민물고기인 가물치에 영감을 받아 탄생한 충청북도 충주의 증류주,5.0,11,증류식소주,25%,375ml,...,깔끔한 술맛을 돋보이게 해줄 담백한 흰살생선 요리와 드셔보세요,상압 중탕 방식으로 증류해 6개월의 옹기 숙성까지 거쳐요,"흰살생선회,삼치구이,게살 계란탕",-1,-1,3,-1,-1,"sashimi,grilled_fish,egg_soup",0.447214
218,takju,담은 [3병/6병/10병],https://www.sooldamhwa.com/damhwaMarket/detail...,images/takju_1.jpg,마음을 담은 한잔,4.8,562,탁주,6.50%,750ml,...,매콤한 음식과 즐겨보세요!,생쌀을 발효한 프리미엄 막걸리,"닭볶음탕,돼지고기 김치찜,떡볶이",4,2,3,0,1,"dakbokkeumtang,kimchijjim,tteokbokki",0.447214


##해당 기준의 결과값 2번째를 보시면, **0.447214**의 값이 나옵니다.

# 3. [코사인 유사도_Flavor & tastPoint & comment 기준]

In [38]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import scipy.sparse as sp

def find_top_similar_drinks3(input_drink_name, csv_file_path):
    # CSV 파일을 Pandas DataFrame으로 읽기
    df = pd.read_csv(csv_file_path, header=None)  # 컬럼 이름이 없다고 지정

    # 입력된 술 이름으로부터 해당 술의 인덱스 찾기
    try:
        target_index = df[df.iloc[:, 1] == input_drink_name].index[0]
    except IndexError:
        return "술 이름을 찾을 수 없습니다."

    # 입력된 술의 텍스트 데이터 및 숫자 데이터 추출
    input_text_data = ' '.join(df.iloc[target_index, [4, 10]].astype(str))
    input_numeric_data = df.iloc[target_index, 14:19].values
    print(input_text_data)
    print(input_numeric_data)

    # 입력된 술을 데이터에서 제외
    df = df.drop(index=target_index)

    # TfidfVectorizer를 사용하여 텍스트 데이터(인덱스 4, 10)를 벡터화
    tfidf_vectorizer = TfidfVectorizer()
    text_data = df.iloc[:, [4, 10]].astype(str).apply(lambda x: ' '.join(x), axis=1)
    tfidf_matrix_text = tfidf_vectorizer.fit_transform(text_data)

    # 숫자 데이터(인덱스 14부터 18)를 분리
    numeric_data = df.iloc[:, 14:19].values

    # TfidfVectorizer로 벡터화된 텍스트 데이터와 숫자 데이터를 병합
    tfidf_matrix = tfidf_matrix_text

    # 숫자 데이터를 스케일링
    scaler = MinMaxScaler()
    scaled_numeric_data = scaler.fit_transform(numeric_data)
    scaled_input_numeric_data = scaler.transform(input_numeric_data.reshape(1, -1))

    # 스케일된 숫자 데이터를 텍스트 데이터와 결합
    combined_data = sp.hstack([tfidf_matrix, sp.csr_matrix(scaled_numeric_data)], format='csr')  # scipy.sparse.hstack 사용

    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(combined_data, combined_data)

    # 입력값과 다른 데이터의 유사도 계산
    input_data_vector = sp.hstack([tfidf_vectorizer.transform([input_text_data]), sp.csr_matrix(scaled_input_numeric_data)], format='csr')
    similarities = cosine_similarity(input_data_vector, combined_data)

    # 가장 높은 유사도를 가진 술 2개의 인덱스 찾기
    similar_indices = similarities[0].argsort()[::-1][:2]

    # 가장 유사도가 높은 2개 술의 인덱스와 유사도 계산
    top_similar_drinks_indices = similar_indices
    top_similar_drinks_data = df.iloc[top_similar_drinks_indices]
    top_similar_drinks_similarities = [similarities[0][index] for index in top_similar_drinks_indices]

    # 결과 데이터에 유사도 추가
    top_similar_drinks_data['유사도'] = top_similar_drinks_similarities

    return top_similar_drinks_data


In [39]:
input_drink_name = '감사'
csv_file_path = '/content/drive/MyDrive/CJ올리브네트웍스 해커톤/DATA/술담화/CSV/alcohol_with_new_category_en.csv'

find_top_similar_drinks3(input_drink_name, csv_file_path)

소중한 분들게 감사한 마음을 담아 달콤한 카스테라가 떠오르는 부드러운 술이에요
[4 3 2 -1 3]


<ipython-input-38-d1ba74a6939f>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_similar_drinks_data['유사도'] = top_similar_drinks_similarities


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,유사도
218,takju,담은 [3병/6병/10병],https://www.sooldamhwa.com/damhwaMarket/detail...,images/takju_1.jpg,마음을 담은 한잔,4.8,562,탁주,6.50%,750ml,...,매콤한 음식과 즐겨보세요!,생쌀을 발효한 프리미엄 막걸리,"닭볶음탕,돼지고기 김치찜,떡볶이",4,2,3,0,1,"dakbokkeumtang,kimchijjim,tteokbokki",0.710313
283,takju,해창 12%,https://www.sooldamhwa.com/damhwaMarket/detail...,images/takju_46.jpg,해창주조장의 대표 막걸리,4.2,10,탁주,12%,900ml,...,술만큼이나 강한 풍미가 있는 기름진 음식과 함께해 보세요,감미료 없이 천천히 시간을 들여 빚은 술,"떡갈비,육회,갈비 구이",3,4,4,0,2,"galbi,beef_tartare,galbi",0.698589


##해당 기준의 결과값 2번째를 보시면, **0.698589**의 값이 나옵니다.

#결과


##[숫자, 텍스트, 숫자 + 텍스트] 3가지 기준으로 유사도의 값이 정확한 판단 기준이 될 수 있지않지만..혜빈님의 결정을 돕고자 생각해본 방안입니다.. 참고로만 생각해주세요 :)